In [1]:
from Functions_6 import normalize_data, time_series_CV_split, wrapper_feature_selector, train_and_predict, fine_tune_alpha, warn
import pandas as pd
from pandas import DataFrame
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, make_scorer
from sklearn.preprocessing import MinMaxScaler 
from time import time
from math import sqrt
from sklearn.linear_model import Lasso
from matplotlib import pyplot
import plotly.offline as py
import plotly.graph_objs as go
import warnings
warnings.warn = warn
py.init_notebook_mode(connected=True)
%matplotlib inline

# Train Set
train_data = pd.read_csv('/Users/Andy/Desktop/Bitcoin/Data/Train_data_after_EDA.csv')
train_data = train_data.drop(columns=['date'])
print('n_features:', len(train_data.iloc[0]))
print('n_samples:', len(train_data))
train_data.head()

/anaconda3/lib/python3.5/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


n_features: 10
n_samples: 365


,close,volume,BCHAIN-DIFF,BCHAIN-AVBLS,BCHAIN-MIREV,BCHAIN-CPTRA,BCHAIN-NTRAN,BCHAIN-HRATE,BCHAIN-CPT,BCHAIN-NTRBL
0,13657.2,10291200000,1.870000e+12,1.037057,35435185.86,125.970638,290422,15177350.25,122.012747,1781.730061
1,14982.1,16846600192,1.920000e+12,1.043383,32334328.99,138.227164,241757,14975580.96,133.747230,1549.724359
2,15201.0,16871900160,1.930000e+12,1.041368,40553327.64,122.635624,340980,16415540.67,118.931690,1994.035088
3,15599.2,21783199744,1.930000e+12,1.065513,39612658.08,103.108719,395963,15071578.27,100.041312,2522.057325
4,17429.5,23840899072,1.930000e+12,1.065833,42527795.29,102.933856,425008,16127548.73,100.063517,2529.809524


In [2]:
# Test Set
test_data = pd.read_csv('/Users/Andy/Desktop/Bitcoin/Data/Test_data_after_EDA.csv')
test_data = test_data.drop(columns=['date'])
print('n_features:', len(test_data.iloc[0]))
print('n_samples:', len(test_data))
test_data.head()



n_features: 10
n_samples: 181


,close,volume,BCHAIN-DIFF,BCHAIN-AVBLS,BCHAIN-MIREV,BCHAIN-CPTRA,BCHAIN-NTRAN,BCHAIN-HRATE,BCHAIN-CPT,BCHAIN-NTRBL
0,3843.52,4324200990,5.250000e+12,0.888394,7406437.589,28.520962,259684,43291796.76,28.520962,1675.380645
1,3943.41,5244856836,5.620000e+12,0.801779,7030739.129,29.953090,234725,41615985.27,29.953090,1575.335570
2,3836.74,4530215219,5.620000e+12,0.947861,7368988.356,27.122182,271696,42174589.10,27.122182,1799.311258
3,3857.72,4847965467,5.620000e+12,0.966222,7564081.637,25.991979,291016,43291796.76,25.991979,1877.522581
4,3845.19,5137609824,5.620000e+12,0.959199,7194486.330,25.533007,281772,41615985.27,25.533007,1891.087248


In [3]:
combined_data = pd.concat([train_data, test_data], ignore_index=True) # use data from 01/2018 onwards only - best stationarity
combined_data['Price'] = combined_data['close'].shift(-1) # Dependent variable Y
combined_data = combined_data.iloc[:-1,1:] # remove the last row and the first close 
print('n_features:', len(combined_data.iloc[0]))
print('n_samples:', len(combined_data))
combined_data.head()


n_features: 10
n_samples: 545


,volume,BCHAIN-DIFF,BCHAIN-AVBLS,BCHAIN-MIREV,BCHAIN-CPTRA,BCHAIN-NTRAN,BCHAIN-HRATE,BCHAIN-CPT,BCHAIN-NTRBL,Price
0,10291200000,1.870000e+12,1.037057,35435185.86,125.970638,290422,15177350.25,122.012747,1781.730061,14982.1
1,16846600192,1.920000e+12,1.043383,32334328.99,138.227164,241757,14975580.96,133.747230,1549.724359,15201.0
2,16871900160,1.930000e+12,1.041368,40553327.64,122.635624,340980,16415540.67,118.931690,1994.035088,15599.2
3,21783199744,1.930000e+12,1.065513,39612658.08,103.108719,395963,15071578.27,100.041312,2522.057325,17429.5
4,23840899072,1.930000e+12,1.065833,42527795.29,102.933856,425008,16127548.73,100.063517,2529.809524,17527.0


In [4]:
# Feature Meta Subset
subset = [8,7,6,5,4,3,2,1,0]

# Split train data into X (features) and Y (dependent variable)
data = combined_data.values 
Y_train = data[0:-181,-1].reshape(-1,1) 
X_train = data[0:-181,:-1]

# Training Validation samples size (1/6/18 - 31/12/18)
n_validation = 214

# Fine-tune alpha
best_alpha = fine_tune_alpha(X_train[:,subset],Y_train,'Lasso')
print('Best alpha:',str(best_alpha))

# Feature Selection
selected_features = wrapper_feature_selector(X_train,Y_train,Lasso(alpha=best_alpha),subset)[0]   
print('Selected Features:',str(selected_features))

Best alpha: 1e-07
Selected Features: [5, 3, 2, 1, 0]


In [5]:
%%time
rmse,Y_train_test,Y_train_pred= train_and_predict(X_train[:,selected_features],Y_train,Lasso(alpha=best_alpha),predict=False)
print('Train RMSE: {:0.2f}'.format(rmse))


Train RMSE: 638.55
CPU times: user 110 ms, sys: 1.58 ms, total: 112 ms
Wall time: 111 ms


In [6]:
%%time
# Feature Meta Subset
subset = [8,7,6,5,4,3,2,1,0]

# Split train data into X (features) and Y (dependent variable)
data = combined_data.values 
Y_train = data[0:-181,-1].reshape(-1,1) 
X_train = data[0:-181,:-1]

# Test samples size (01/01/19 - 30/06/19)
n_validation = 181

# Split test data into X (features) and Y (dependent variable)
Y_test = data[:,-1].reshape(-1,1) # including train data for fitting the model
X_test = data[:,:-1]



CPU times: user 224 µs, sys: 11 µs, total: 235 µs
Wall time: 224 µs


In [59]:
len(Y_test)

545

In [11]:
rmse,Y_test,Y_pred= train_and_predict(X_test,Y_test,Lasso(alpha=1e-07),n_validation,subset,predict=True)
np.savetxt('Lasso.csv', Y_pred.reshape(-1,1), delimiter=',',  fmt='%1.10e')
print('Test RMSE: {:0.2f}'.format(rmse))

Test Sample 1 - RMSE: 618.96, Selected Features: [5, 3, 2, 1, 0]
Test Sample 2 - RMSE: 249.32, Selected Features: [8, 7, 5, 4, 3, 2, 1, 0]
Test Sample 3 - RMSE: 765.67, Selected Features: [8, 6, 3, 2, 1, 0]
Test Sample 4 - RMSE: 473.97, Selected Features: [8, 7, 6]
Test Sample 5 - RMSE: 150.37, Selected Features: [8, 7, 5, 4, 3, 2, 1, 0]
Test Sample 6 - RMSE: 66.15, Selected Features: [8, 5, 3, 2, 1, 0]
Test Sample 7 - RMSE: 121.20, Selected Features: [8, 6, 5, 4, 3, 2, 1]
Test Sample 8 - RMSE: 232.46, Selected Features: [8, 7, 5, 4, 3, 2]
Test Sample 9 - RMSE: 153.63, Selected Features: [7, 5, 4, 3, 2, 1]
Test Sample 10 - RMSE: 229.19, Selected Features: [5, 3, 2, 1, 0]
Test Sample 11 - RMSE: 119.81, Selected Features: [8, 7, 6, 5, 4, 3, 2, 0]
Test Sample 12 - RMSE: 39.18, Selected Features: [8, 7, 5, 4, 3, 2]
Test Sample 13 - RMSE: 162.88, Selected Features: [7, 6, 5, 3, 2, 1, 0]
Test Sample 14 - RMSE: 14.17, Selected Features: [5, 3, 2, 1, 0]
Test Sample 15 - RMSE: 93.54, Selected F

ValueError: Found array with 0 feature(s) (shape=(408, 0)) while a minimum of 1 is required by MinMaxScaler.

In [ ]:
# Output prediction to csv file
output = {'Y_pred': Y_pred.reshape(-1,)}
output = DataFrame(output)
output.to_csv('Lasso.csv', index=False)


In [ ]:
trace1 = go.Scatter(
    x = np.arange(0, len(Y_pred), 1),
    y = Y_pred.reshape(-1,),
    mode = 'lines',
    name = 'Predicted labels',
    line = dict(color=('rgb(244, 146, 65)'), width=2)
)
trace2 = go.Scatter(
    x = np.arange(0, len(Y_test), 1),
    y = Y_test.reshape(-1,),
    mode = 'lines',
    name = 'True labels',
    line = dict(color=('rgb(66, 244, 155)'), width=2)
)

layout = dict(title = 'Comparison of true prices (on the test dataset) with prices our Lasso model predicted',
             xaxis = dict(title = 'Day number'), yaxis = dict(title = 'Price, USD'))
fig = dict(data=[trace1, trace2], layout=layout)
py.iplot(fig, filename='results_demonstrating0')

print('Test RMSE: {:0.2f}'.format(rmse))
